# Hyperparameter Tuning

Hyperparameter Tuning uses grid search to scan through a given hyperparameter space for a model and find out the best combination of hyperparameters with respect to a given performance metric.

Importing the necessary modules and reading the dataset.

Set up the environment and install TurboML's SDK. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
! conda install -qq conda-forge::libstdcxx-ng anaconda::protobuf
! gdown -q 1UG8cmBLdYtEVJZG8Rgz454l722VAt_zE
! unzip -qq turboml.zip -d turboml && pip install -qq turboml/*.whl && rm -rf turboml turboml.zip

Login to your TurboML instance.

In [ ]:
import pandas as pd
import turboml as tb
tb.init(backend_url=BACKEND_URL, api_key=API_KEY)
from sklearn import metrics

In [ ]:
transactions_df = pd.read_csv("data/transactions.csv").reset_index()
labels_df = pd.read_csv("data/labels.csv").reset_index()

## Dataset

We use the `PandasDataset` class to create a dataset to be used for tuning.

For this example, we use the first 100k rows.

In [ ]:
transactions_100k = tb.PandasDataset(
    dataframe=transactions_df[:100000], key_field="index", streaming=False
)
labels_100k = tb.PandasDataset(
    dataframe=labels_df[:100000], key_field="index", streaming=False
)

In [ ]:
numerical_fields = ["transactionAmount", "localHour"]
categorical_fields = ["digitalItemCount", "physicalItemCount", "isProxyIP"]
inputs = transactions_100k.get_input_fields(
    numerical_fields=numerical_fields, categorical_fields=categorical_fields
)
label = labels_100k.get_label_field(label_field="is_fraud")

## Training/Tuning

We will be using the `AdaBoost Classifier` with `Hoeffding Tree Classifier`  being the base model as an example.

In [ ]:
model_to_tune = tb.AdaBoostClassifier(
    n_classes=2, base_model=tb.HoeffdingTreeClassifier(n_classes=2)
)

Since a particular model object can include other base models and PreProcessors as well, the `hyperparameter_tuning` function accepts a list of hyperparameter spaces for all such models as part of the `model` parameter, and tests all possible combinations across the different spaces.

In this example, the first dictionary in the list corresponds to the hyperparameters of `AdaBoostClassifier` while the second dictionary is the hyperparameter space for the `HoeffdingTreeClassifier`. 

It is not necessary to include all possible hyperparameters in the space; default values are taken for those not specified 

In [ ]:
model_score_list = tb.hyperparameter_tuning(
    metric_to_optimize="accuracy",
    model=model_to_tune,
    hyperparameter_space=[
        {"n_models": [2, 3]},
        {
            "delta": [1e-7, 1e-5, 1e-3],
            "tau": [0.05, 0.01, 0.1],
            "grace_period": [200, 100, 500],
            "n_classes": [2],
            "leaf_pred_method": ["mc"],
            "split_method": ["gini", "info_gain", "hellinger"],
        },
    ],
    input=inputs,
    labels=label,
)
best_model, best_score = model_score_list[0]
best_model

In [ ]:
transactions_full = tb.PandasDataset(
    dataframe=transactions_df, key_field="index", streaming=False
)
features = transactions_full.get_input_fields(
    numerical_fields=numerical_fields, categorical_fields=categorical_fields
)

outputs = best_model.predict(features)

In [ ]:
print(
    "Accuracy: ",
    metrics.accuracy_score(labels_df["is_fraud"], outputs["predicted_class"]),
)
print("F1: ", metrics.f1_score(labels_df["is_fraud"], outputs["predicted_class"]))